In [1]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`
┌ Info: Precompiling FUSE [e64856f0-3bb8-4376-b4b7-c03396503992]
└ @ Base loading.jl:1317


In [2]:
time_slice = IMAS.equilibrium__time_slice()
profiles_1d = time_slice.profiles_1d

n=11
profiles_1d.psi = collect(range(0.0, 1.0, length=n))
profiles_1d.pressure = collect(1.0.-range(0.0, 1.0, length=n).^2)
profiles_1d.volume = collect(range(0.0, 1.0, length=n))
time_slice.global_quantities.energy_mhd

0.9975

In [3]:
core_profiles = IMAS.core_profiles();
resize!(core_profiles.profiles_1d,1)
profiles_1d = core_profiles.profiles_1d[1]

profiles_1d.grid.rho_tor_norm = collect(range(0.0, 1.0, length=21))
profiles_1d.electrons.density = (1.0 .- profiles_1d.grid.rho_tor_norm.^2);
profiles_1d.electrons.temperature = (x;_...) -> (1.0 .- x.^2);

println(profiles_1d.electrons.pressure)
profiles_1d.grid.rho_tor_norm = collect(range(0.0, 1.0, length=5))
println(profiles_1d.electrons.temperature)

[1.60218e-19, 1.594179113625e-19, 1.570296618e-19, 1.5308930036250002e-19, 1.476569088e-19, 1.408166015625e-19, 1.326765258e-19, 1.2336886136250001e-19, 1.1304982079999998e-19, 1.018996493625e-19, 9.0122625e-20, 7.79470583625e-20, 6.56252928e-20, 5.343370436249998e-20, 4.16727018e-20, 3.06667265625e-20, 2.0764252799999985e-20, 1.2337787362500006e-20, 5.783869799999997e-21, 1.5230723625000009e-21, 0.0]
[1.0, 0.9375, 0.75, 0.4375, 0.0]


In [4]:
time_slice = IMAS.equilibrium__time_slice()
profiles_1d = time_slice.profiles_1d

using ForwardDiff
using Zygote

n=11
profiles_1d.psi = collect(range(0.0, 1.0, length=n))
profiles_1d.pressure = p0 = collect(1.0.-range(0.0, 1.0, length=n).^2)
profiles_1d.volume = collect(range(0.0, 1.0, length=n))

function test_gradient(mul)
    profiles_1d.pressure = profiles_1d.pressure.*mul.^2
    sum(time_slice.global_quantities.energy_mhd)
end

#test_gradient(3)
ForwardDiff.derivative(test_gradient, 2.0)
#Zygote.gradient(test_gradient, 2.0)

3.99

In [6]:
filename = joinpath(dirname(dirname(abspath(@__FILE__))), "sample", "sample_equilibrium_ods.json")
data  = IMAS.json2imas(filename; verbose=false)

EQUILIBRIUM
｜time_slice
｜｜[1]
｜｜｜time ➡ Float64
｜｜｜profiles_1d
｜｜｜｜b_field_max ➡ 17-element Vector{Real}
｜｜｜｜dvolume_drho_tor ➡ 17-element Vector{Real}
｜｜｜｜gm9 ➡ 17-element Vector{Real}
｜｜｜｜dpsi_drho_tor ➡ 17-element Vector{Real}
｜｜｜｜surface ➡ 17-element Vector{Real}
｜｜｜｜rho_tor ➡ 17-element Vector{Real}
｜｜｜｜b_field_min ➡ 17-element Vector{Real}
｜｜｜｜darea_dpsi ➡ 17-element Vector{Real}
｜｜｜｜squareness_upper_inner ➡ 17-element Vector{Real}
｜｜｜｜squareness_lower_inner ➡ 17-element Vector{Real}
｜｜｜｜rho_tor_norm ➡ 17-element Vector{Real}
｜｜｜｜elongation ➡ 17-element Vector{Real}
｜｜｜｜b_field_average ➡ 17-element Vector{Real}
｜｜｜｜psi ➡ 17-element Vector{Real}
｜｜｜｜gm8 ➡ 17-element Vector{Real}
｜｜｜｜dpressure_dpsi ➡ 17-element Vector{Real}
｜｜｜｜triangularity_upper ➡ 17-element Vector{Real}
｜｜｜｜darea_drho_tor ➡ 17-element Vector{Real}
｜｜｜｜area ➡ 17-element Vector{Real}
｜｜｜｜trapped_fraction ➡ 17-element Vector{Real}
｜｜｜｜volume ➡ 17-element Vector{Real}
｜｜｜｜dvolume_dpsi ➡ 17-element Vector{Real}
｜｜｜｜f

type
｜｜｜｜｜name ➡ 10-codeunit String
｜｜｜｜｜description ➡ 10-codeunit String
｜｜｜｜｜index ➡ Int64
｜｜｜｜unit
｜｜｜｜｜[1]
｜｜｜｜｜｜outline
｜｜｜｜｜｜｜r ➡ 86-element Vector{Real}
｜｜｜｜｜｜｜z ➡ 86-element Vector{Real}

DATASET_DESCRIPTION
｜data_entry
｜｜pulse ➡ Int64


In [26]:
Base.ndims(data::Array{T,N} where T <: Real where N<:Integer) = data.N
